In [1]:
#Requirements for Chat_Bot implementation:
# 1. Keras(Deep Learning library), 2. NLTK(Natural Language Processing toolkit), and 3. some helpful libraries. 


In [2]:
#Step 1(a). Import Libraries

import numpy as np
import random
import nltk
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import json
import pickle

Using TensorFlow backend.


In [3]:
#Step 1(b). Load the data
lemmatizer = WordNetLemmatizer()
intents_file = open('D:DS_TriS/Chat_bot/intents.json').read()
intents = json.loads(intents_file)

In [4]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [5]:
#Step 2. Preprocessing the data

#The model cannot take the raw data. It has to go through a lot of pre-processing for the machine to easily understand.
#For textual data, there are many preprocessing techniques available. The first technique is tokenizing.
#By observing the intents file, we can see that each tag contains a list of patterns and responses. 
#We tokenize each pattern and add the words in a list.

words = []
classes = []
documents = []
ignore_letters =['!','?',',','.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in corpus
        documents.append((word, intent['tag']))
        #add to our classes ;ist
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(words,len(words))
print(documents, len(documents))
print(classes, len(classes))

['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'you', 'can', 'do', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'What', 'support', 'is', 'offered', 'How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?', 'Open', 'adverse', 'drugs', 'module', 'Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', 'adverse', 'behavior', 'List', 'all', 'drugs', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction', 'Which', 'drugs', 'dont', 'have', 'adverse', 'reaction', '?', 'Open', 'blood', 'pressure', 'module', 'Task', 'related', 'to', 'blood', 'pressure', 'Blood', 'pressure', 'data', 'entry', 'I', 'want', 'to', 'log', 'blood', 'pr

In [6]:
#Another technique is Lemmatization. We can convert words into the lemma form so that we can reduce all the canonical words.

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
#sort classes
classes = sorted(list(set(classes)))

#documents = combination between pattern s and intents
print(len(documents),"documents")

#classes = intents
print(len(classes),"classes",classes)

#words = all words, vocublary
print(len(word),"unique lemmatized words", words)

pickle.dump(words, open('D:DS_TriS/Chat_bot/words.pkl','wb'))
pickle.dump(classes,open('D:DS_TriS/Chat_bot/classes.plk','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
4 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [7]:
#Step 3. Create training data

#Create the training data
training = []
#create empty array for the output
output_empty = [0]*len(classes)

#training set, bag of words for every sentance
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenize words for pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represented related word
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    #create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    #Output is a '0' for each tag and '1' for current tag(for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    
#Shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)

#Create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print(train_x)
print(train_y)
print("Training data created")

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
# Step 4. Traiing the model

#deep neural networks model
model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),),activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

#Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr = 0.1, decay = 1e-6, momentum=0.9,nesterov= True)
model.compile(loss='categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

#Training and Saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5,verbose = 1)
model.save('D:DS_TriS/Chat_bot/chatbot_model.h5', hist)
print("Model is created")

Epoch 1/200
47/47 [==============================] - 0s 5ms/step - loss: 2.1794 - accuracy: 0.1489
Epoch 2/200
47/47 [==============================] - 0s 596us/step - loss: 1.8501 - accuracy: 0.3191
Epoch 3/200
47/47 [==============================] - 0s 766us/step - loss: 2.1463 - accuracy: 0.4255
Epoch 4/200
47/47 [==============================] - 0s 596us/step - loss: 1.9620 - accuracy: 0.4255
Epoch 5/200
47/47 [==============================] - 0s 766us/step - loss: 2.2574 - accuracy: 0.4255
Epoch 6/200
47/47 [==============================] - 0s 766us/step - loss: 3.4102 - accuracy: 0.3830
Epoch 7/200
47/47 [==============================] - 0s 851us/step - loss: 10.4226 - accuracy: 0.5106
Epoch 8/200
47/47 [==============================] - 0s 680us/step - loss: 28.9587 - accuracy: 0.2340
Epoch 9/200
47/47 [==============================] - 0s 851us/step - loss: 118.1952 - accuracy: 0.1915
Epoch 10/200
47/47 [==============================] - 0s 681us/step - loss: 16198.4393 - 

47/47 [==============================] - 0s 596us/step - loss: 2.2196 - accuracy: 0.1489
Epoch 82/200
47/47 [==============================] - 0s 681us/step - loss: 2.2316 - accuracy: 0.1489
Epoch 83/200
47/47 [==============================] - 0s 766us/step - loss: 2.1823 - accuracy: 0.1702
Epoch 84/200
47/47 [==============================] - 0s 596us/step - loss: 2.1835 - accuracy: 0.1064
Epoch 85/200
47/47 [==============================] - 0s 681us/step - loss: 2.2145 - accuracy: 0.1489
Epoch 86/200
47/47 [==============================] - 0s 681us/step - loss: 2.2248 - accuracy: 0.1489
Epoch 87/200
47/47 [==============================] - 0s 681us/step - loss: 2.2160 - accuracy: 0.1489
Epoch 88/200
47/47 [==============================] - 0s 681us/step - loss: 2.2293 - accuracy: 0.1064
Epoch 89/200
47/47 [==============================] - 0s 681us/step - loss: 2.2259 - accuracy: 0.0638
Epoch 90/200
47/47 [==============================] - 0s 681us/step - loss: 2.2260 - accuracy: 

47/47 [==============================] - 0s 596us/step - loss: 2.2200 - accuracy: 0.0851
Epoch 160/200
47/47 [==============================] - 0s 511us/step - loss: 2.2416 - accuracy: 0.0213
Epoch 161/200
47/47 [==============================] - 0s 595us/step - loss: 2.2295 - accuracy: 0.0426
Epoch 162/200
47/47 [==============================] - 0s 511us/step - loss: 2.2310 - accuracy: 0.1489
Epoch 163/200
47/47 [==============================] - 0s 596us/step - loss: 2.2204 - accuracy: 0.1064
Epoch 164/200
47/47 [==============================] - 0s 680us/step - loss: 2.2268 - accuracy: 0.1064
Epoch 165/200
47/47 [==============================] - 0s 681us/step - loss: 2.1787 - accuracy: 0.1702
Epoch 166/200
47/47 [==============================] - 0s 596us/step - loss: 2.1759 - accuracy: 0.1702
Epoch 167/200
47/47 [==============================] - 0s 596us/step - loss: 2.2265 - accuracy: 0.1489
Epoch 168/200
47/47 [==============================] - 0s 681us/step - loss: 2.2211 - a